In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
# Clear Database and create connection
try:
    os.remove('../HomelessData.db')
    print("Cleared Database")
except FileNotFoundError : 
    pass
conn = sqlite3.connect('../HomelessData.db')
c = conn.cursor()

Cleared Database


In [3]:
# Create ShelterAvalibility dataframe and database
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]: 
    try: 
        hic_df = pd.read_excel("../resources/2007-2020-HIC-Counts-by-CoC.xlsx",sheet,skiprows=[0],skipfooter=1)

        # Adjust Column
        hic_df.rename(columns = {'CoC Number' : 'Coc','Total Year-Round Beds (ES, TH, SH)' : 'Total','Total Year-Round Beds (ES)' : 'Emergency','Total Year-Round Beds (TH)' : 'Transitional','Total Year-Round Beds (SH)' : 'SafeHaven','Total Year-Round Beds (RRH)' : 'RapidRehousing','Total Year-Round Beds (PSH)' : 'PermanentSupportive','Total Year-Round Beds (OPH)' : 'PermanentOther'},inplace=True)
        hic_df = hic_df[["Coc", "Total", "Emergency", "Transitional", "SafeHaven", "RapidRehousing", "PermanentSupportive", "PermanentOther"]]
        hic_df["Year"] = sheet
        hic_df["CocYear"] = hic_df["Coc"] +" "+ hic_df["Year"]

        # Insert Into DB
        hic_df.to_sql("shelterAvalibility",con=conn,if_exists = 'append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E:
        print(f"Couldnt get data from {sheet}")
print("Finished")

Gathered Data from 2020
Gathered Data from 2019
Gathered Data from 2018
Gathered Data from 2017
Gathered Data from 2016
Gathered Data from 2015
Gathered Data from 2014
Couldnt get data from 2013
Finished


C:\Users\Jonat\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
# Create PIT and coc dataframe and database
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]: 
    try:
        pit_df = pd.read_excel("../resources/2007-2020-PIT-Estimates-by-CoC.xlsx",sheet,skipfooter=3)

        # Rename pit_df columns
        pit_df.rename(columns = {'CoC Number' : 'Coc',f'Overall Homeless, {sheet}' : "Total",f'Unsheltered Homeless, {sheet}' : "Unsheltered"},inplace=True)

        # Create New Columns
        pit_df["Sheltered"] = pit_df["Total"]-pit_df["Unsheltered"]
        pit_df = pit_df[['Coc','Total','Unsheltered','Sheltered']]
        pit_df["Year"] = sheet
        pit_df["CocYear"] = pit_df["Coc"] +" "+ pit_df["Year"]

        # Create HomelessData table
        pit_df.to_sql("HomelessCounts",con=conn,if_exists='append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E : 
        print(f"Couldnt get data from {sheet}")
print("Finished")

Gathered Data from 2020
Gathered Data from 2019
Gathered Data from 2018
Gathered Data from 2017
Gathered Data from 2016
Gathered Data from 2015
Gathered Data from 2014
Gathered Data from 2013
Finished


In [6]:
# Create County Data Table
county_data_df = pd.read_csv("../Resources\county_level_data\county_data.csv")
clean_df = pd.DataFrame()
for column in ["2019","2018","2017","2016","2015","2014","2013"]: 
    df = county_data_df[["FIPS",f"{column}_population",f"Unemployment_rate_{column}"]]
    df.rename(columns = {f"{column}_population" : "Population",f"Unemployment_rate_{column}" : "Unemploment"},inplace=True)
    df["year"] = column
    df.to_sql("CountyData",if_exists = "append",con=conn,index=False)
    print(f"Gathered Data from {column}")
print("Finished")

Gathered Data from 2019
Gathered Data from 2018
Gathered Data from 2017
Gathered Data from 2016
Gathered Data from 2015
Gathered Data from 2014
Gathered Data from 2013
Finished


C:\Users\Jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-6-d9986782b0d0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"] = column


In [8]:
# Create CountyMeta
county_meta_df = pd.read_csv("../Resources\county_level_data\county_coc_source.csv")
lat_lng_df = pd.read_csv("../Resources\county_level_data\wiki_county_lat_long.csv",skiprows = [1])[["FIPS","Latitude","Longitude"]]
df = pd.merge(county_meta_df,lat_lng_df, on="FIPS")
df.rename(columns = {"number" : "Coc"},inplace=True)
df.to_sql("CountyMeta",con=conn,index=False)

In [10]:
# Transform County Data into Coc data file
county_df = pd.read_sql("SELECT * FROM CountyData",con=conn)
county_df["Population"] = county_df["Population"].str.replace(",","")
county_df["Population"] = pd.to_numeric(county_df["Population"])

county_meta_df = pd.read_sql("SELECT * FROM CountyMeta",con=conn)

new_county_df = pd.merge(county_df,county_meta_df,on="FIPS")

grouped_df = new_county_df[["Population","Coc","year"]].groupby(["year","Coc"]).sum()
grouped_df["Unemployment"] = new_county_df[["Unemploment","Coc","year"]].groupby(["year","Coc"]).mean()["Unemploment"]
grouped_df.reset_index(inplace=True)

grouped_df["CocYear"] = grouped_df["Coc"] +' ' + grouped_df["year"]

grouped_df.to_sql("CocCountyData",con=conn)

grouped_df

,year,Coc,Population,Unemployment,CocYear
0,2013,AK-500,301142.0,5.200000,AK-500 2013
1,2013,AK-501,418032.0,10.191667,AK-501 2013
2,2013,AL-500,949114.0,5.900000,AL-500 2013
3,2013,AL-501,608803.0,7.700000,AL-501 2013
4,2013,AL-502,209007.0,7.925000,AL-502 2013
...,...,...,...,...,...
2704,2019,WV-500,122691.0,5.325000,WV-500 2019
2705,2019,WV-501,131347.0,4.800000,WV-501 2019
2706,2019,WV-503,264539.0,5.775000,WV-503 2019
2707,2019,WV-508,1273570.0,5.857778,WV-508 2019


In [11]:
# Merging the HIC PIT and County Data into CocData
c.execute("""
CREATE TABLE CocData AS
SELECT * FROM ShelterAvalibility
JOIN HomelessCounts
ON ShelterAvalibility.CocYear = HomelessCounts.CocYear
JOIN CocCountyData
ON ShelterAvalibility.CocYear = CocCountyData.CocYear
""")
c.execute("ALTER TABLE CocData DROP 'CocYear:1';")
c.execute("ALTER TABLE CocData DROP 'Coc:1';")
c.execute("ALTER TABLE CocData DROP 'Year:1';")
c.execute("ALTER TABLE CocData DROP 'Total:1';")

# Drop the old tables
c.execute("DROP TABLE ShelterAvalibility")
c.execute("DROP TABLE HomelessCounts")
c.execute("DROP TABLE CountyData")

pd.read_sql("SELECT * FROM CocData",con=conn)

,Coc,Total,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Year,CocYear,Unsheltered,Sheltered,index,year:2,Coc:2,Population,Unemployment,CocYear:2
0,AK-500,1033,799,234,0,171,601,71,2019,AK-500 2019,97,1014,2322,2019,AK-500,288000.0,4.400000,AK-500 2019
1,AK-501,845,575,270,0,67,386,0,2019,AK-501 2019,176,620,2323,2019,AK-501,425343.0,6.991667,AK-501 2019
2,AL-500,827,516,277,34,106,1740,0,2019,AL-500 2019,326,655,2324,2019,AL-500,965787.0,2.600000,AL-500 2019
3,AL-501,394,261,133,0,31,240,0,2019,AL-501 2019,202,303,2325,2019,AL-501,636444.0,3.250000,AL-501 2019
4,AL-502,189,106,83,0,0,42,8,2019,AL-502 2019,201,189,2326,2019,AL-502,209041.0,3.350000,AL-502 2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,WV-500,117,77,40,0,1,18,0,2014,WV-500 2014,9,88,769,2014,WV-500,129232.0,7.000000,WV-500 2014
2216,WV-501,261,180,68,13,4,403,108,2014,WV-501 2014,10,208,770,2014,WV-501,138156.0,6.050000,WV-501 2014
2217,WV-503,430,302,128,0,2,178,0,2014,WV-503 2014,13,347,771,2014,WV-503,279577.0,7.775000,WV-503 2014
2218,WV-508,1157,804,353,0,66,620,0,2014,WV-508 2014,398,940,772,2014,WV-508,1302524.0,7.444444,WV-508 2014


In [ ]:
# Create The Funding Database and Dataframe
for sheet in ["2020","2019","2018","2017","2016","2015","2014","2013"]:
    try:
        awards_df = pd.read_excel("../resources/2019-Awards-by-CoC-Component-Project.xlsx",sheet)
        awards_df.rename(columns = {"CoC Number" : "CoC"},inplace=True)
        awards_df = awards_df[["CoC","Component","Awarded Amount"]]
        awards_df["Year"] = sheet
        awards_df.to_sql("FundingData",con=conn,if_exists='append',index=False)
        print(f"Gathered Data from {sheet}")
    except Exception as E : 
        print(f"Couldnt gather data from {sheet}")
        
print("Finished")

In [ ]:
# Create the Coc Meta
df = pd.read_excel("../resources/2007-2020-PIT-Estimates-by-CoC.xlsx",skipfooter=3)[['CoC Number',"CoC Category"]]
df.to_sql("CocMeta",con=conn,index=False)